In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import missingno as msno
from pathlib import Path
from tqdm import tqdm
import wandb
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
train=pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test=pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import KFold
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
train["file_path"] = train["Id"].apply(lambda x: "../input/petfinder-pawpularity-score/train/" + x + ".jpg")
test["file_path"] = test["Id"].apply(lambda x: "../input/petfinder-pawpularity-score/test/" + x + ".jpg")

In [ ]:
def Normalize(image):
        maxvalue = np.max(image)  # m_Voxel의 가장 큰 값을 뽑아낸다.
        image = image/maxvalue  # 0.~으로 정규화 된다. 
        return image

In [ ]:
image_size = 128
def preprocess(image_url):
    image = cv2.imread(image_url)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = tf.cast(image, tf.float32)
    image = Normalize(image)
#     image = tf.image.central_crop(image, 1.0)
    image = tf.image.resize(image, (image_size, image_size))
    return image
x_train=[]
for i in train['file_path']:
    x1=preprocess(i)
    x_train.append(x1)

In [ ]:
test2=[]
for i in test['file_path']:
    x1=preprocess(i)
    test2.append(x1)
test2=np.array(test2)

In [ ]:
x_train=np.array(x_train)
y_train=train['Pawpularity']
x_train,x_test,y_train,y_test=train_test_split(x_train,y_train,test_size=0.2)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=4,kernel_size=3,strides=2,padding='same',activation='relu'),
    tf.keras.layers.Conv2D(filters=8,kernel_size=3,strides=2,padding='same',activation='relu'),
    tf.keras.layers.Conv2D(filters=16,kernel_size=3,strides=2,padding='same',activation='relu'),
    tf.keras.layers.Conv2D(filters=32,kernel_size=3,strides=2,padding='same',activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation = "sigmoid"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation = "sigmoid"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation = "sigmoid"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1)
])

optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001)
rmse_loss = tf.keras.metrics.RootMeanSquaredError(name="rmse")

model.compile(optimizer = optimizer, loss = 'mse', metrics=[rmse_loss, "mae", "mape"])

In [ ]:
early_stopping = EarlyStopping(monitor = 'mse', min_delta = 0.0001, patience = 50, verbose =1, mode = 'auto', baseline = None, restore_best_weights = True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(factor=0.001,patience=30, min_lr=0.00001)
callbacks = [early_stopping,reduce_lr]

In [ ]:
history = model.fit(x_train,y_train, epochs=1000,batch_size=32,validation_data = (x_test,y_test),callbacks=callbacks)

In [ ]:
cnn_pred=model.predict(test2)

In [ ]:
cnn=pd.DataFrame()
cnn['Id']=test['Id']
cnn['Pawpularity']=cnn_pred
cnn.to_csv('submission.csv',index=False)